In [ ]:
__author__ = 'Gaurav Chachra'
__email__ = 'chachra@berkeley.edu'

from keras import models
import numpy as np
import cv2
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import urllib.request as req
import os
import urllib
import tweepy
import json

In [ ]:
model = models.load_model('../model/best_model.h5')

In [ ]:
def twitter_image(url):
    X_any = []

    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype='uint8')
    img_temp = cv2.resize(cv2.imdecode(image, cv2.IMREAD_COLOR), (150, 150), interpolation=cv2.INTER_CUBIC)
    b,g,r = cv2.split(img_temp)           # get b, g, r
    rgb_img1 = cv2.merge([r,g,b])     # switch it to r, g, b
    X_any = []
    X_any.append(rgb_img1)
    return X_any

def image_classify(url, tweet_id):
    image = twitter_image(url)
    image_array = np.array(image)/255
    predicted_label = model.predict(image_array).flatten()
    damage_probability = predicted_label[1]
    if damage_probability>=0.5:
        req.urlretrieve(url, '../output/damage/' + str(round(damage_probability, 2)) + '-' + str(tweet_id) + '-' + url.split('/')[-1])
    if damage_probability<0.5:
        req.urlretrieve(url, '../output/other/' + str(round(damage_probability, 2)) + '-' + str(tweet_id) + '-' + url.split('/')[-1])
#         req.urlretrieve(url, '../output/other/' + str(damage_probability) + '_' + str(tweet_id) + url[-4:])

In [ ]:
if 'tweets.pkl' in os.listdir():
    tweets_df = pd.read_pickle('./tweets.pkl')
else:
    tweets_df = pd.DataFrame(columns = ['id', 'media_url'])
    tweets_df.to_pickle('./tweets.pkl')

class MyStreamListener(tweepy.StreamListener):

    def on_status(self, status):
        
        tweets_df = pd.read_pickle('./tweets.pkl')

        if 'extended_entities' in status._json.keys():
            if 'media' in status._json['extended_entities'].keys():
                for image in status._json['extended_entities']['media']:
                    if image['media_url'][21] == 'm':
                        if tweets_df['media_url'].str.contains(image['media_url']).any() == False:
                            tweets_df = tweets_df.append({'id':status._json['id'], 'media_url':image['media_url']}, ignore_index=True)
                            tweets_df.to_pickle('./tweets.pkl')
                            image_classify(image['media_url'], status._json['id'])


with open('twitter_credentials.json', 'r') as file:
    credentials = json.load(file)
auth = tweepy.OAuthHandler(credentials['CONSUMER_KEY'] , credentials['CONSUMER_SECRET'] )
auth.set_access_token(credentials['ACCESS_TOKEN'], credentials['ACCESS_SECRET'])

api = tweepy.API(auth)

keyword = 'earthquake'
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)
myStream.filter(track=[keyword])